In [2]:

from omegaconf import DictConfig,OmegaConf
conf = OmegaConf.load('config/config.yaml')
pic_=conf['config']['pic_']
tracking_uri_=conf['config']['tracking_uri']

In [3]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pickle5 as pickle
from catboost import Pool
from sklearn.ensemble import StackingClassifier, VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split,StratifiedKFold

from sklearn.metrics import accuracy_score, f1_score,cohen_kappa_score,roc_auc_score,log_loss
import mlflow

In [4]:
##load_feature
fs=[]
with open('ml_output/04_05_modeling/feature_selection/fs.pickle', 'rb') as handle:
    fs_=pickle.load(handle)
    
    catboost_params=fs_['catboost-wo_artificial']
    fs_cat_wo_na=catboost_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_cat_wo_na)
    catboost_params=fs_['catboost-w_artificial']
    fs_cat_w_a=catboost_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_cat_w_a)
    
    lgbm_params=fs_['lightgbm-wo_artificial']
    fs_lgbm_wo_na=lgbm_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_lgbm_wo_na)
    lgbm_params=fs_['lightgbm-w_artificial']
    fs_lgbm_w_a=lgbm_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_lgbm_w_a)
    
    rf_params=fs_['randomforest-wo_artificial']
    fs_rf_wo_na=rf_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_rf_wo_na)
    rf_params=fs_['randomforest-wo_artificial']
    fs_rf_w_a=rf_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_rf_w_a)
    
    fs=list(dict.fromkeys(fs))

In [5]:
##Load model
model=[]
with open('ml_output/04_05_modeling/tuning/model_tuned.pickle', 'rb') as handle:
    model=pickle.load(handle)

In [6]:
##Load data
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +".csv",sep=',')
data_sample=data_sample
X=data_sample[fs]
y=data_sample[['nama_valid']]
    
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)

In [7]:

#sk_fold=StratifiedKFold(n_splits=5,shuffle=False)
experiment_name = "Modeling and Stacking"
    ## check if the experiment already exists
if not mlflow.get_experiment_by_name(experiment_name):
        mlflow.create_experiment(name=experiment_name) 
experiment = mlflow.get_experiment_by_name(experiment_name)
mlflow.set_tracking_uri(tracking_uri_)

In [8]:
def clf_estimate(clf,f_select,params):
    global valid_x, valid_y,experiment
    pred_y=clf.predict(valid_x[f_select])
    preds_proba_y=clf.predict_proba(valid_x[f_select])
    f1_micro_=f1_score(valid_y, pred_y,average='micro')
    log_loss_=log_loss(valid_y,preds_proba_y)
    roc_auc_score_=roc_auc_score(valid_y, preds_proba_y, average="weighted", multi_class="ovr")
    cohen_kappa_score_=cohen_kappa_score(valid_y, pred_y)
    with mlflow.start_run(experiment_id = experiment.experiment_id,
                          run_name=params):
        mlflow.log_metric("f1_score", np.mean(f1_micro_))
        mlflow.log_metric("log_loss", np.mean( log_loss_))
        mlflow.log_metric("roc_auc", np.mean(roc_auc_score_))
        mlflow.log_metric("cohen_kappa", np.mean(cohen_kappa_score_))
        mlflow.log_param('classifier',params)
        if params[:1]=='c':
            mlflow.catboost.log_model(clf, params)
        elif params[:1]=='l':
            mlflow.lightgbm.log_model(clf, params)
        else:
            mlflow.sklearn.log_model(clf,params)

In [9]:
params='catboost_wo_na'
clf_estimate(model[params],fs_cat_wo_na,params)

params='catboost_w_a'
clf_estimate(model[params],fs_cat_w_a,params)

params='lightgbm_wo_na'
clf_estimate(model[params],fs_lgbm_wo_na,params)

params='lightgbm_w_a'
clf_estimate(model[params],fs_lgbm_w_a,params)

params='lightgbm_wo_na'
clf_estimate(model[params],fs_lgbm_wo_na,params)

params='randomforest_wo_na'
clf_estimate(model[params],fs_rf_wo_na,params)

cat_na=model['randomforest_w_a']
clf_estimate(model[params],fs_rf_w_a,params)

/opt/tljh/user/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [10]:
estimators_na = [('catboost_wo_na',model['catboost_wo_na']),
              ('lightgbm_wo_na',model['lightgbm_wo_na']),
              ('randomforest_wo_na',model['randomforest_wo_na'])]
clf_na = StackingClassifier(
    estimators=estimators_na, final_estimator=LogisticRegression(),cv=3
    )
clf_na.fit(train_x,train_y)
clf_estimate(clf_na,fs,'stacking_wo_na')


estimators_a = [('catboost_w_a',model['catboost_w_a']),
              ('lightgbm_w_a',model['lightgbm_w_a']),
              ('randomforest_w_a',model['randomforest_w_a'])]
clf_a = StackingClassifier(
    estimators=estimators_a, final_estimator=LogisticRegression(),cv=3
    )
clf_a.fit(train_x,train_y)
clf_estimate(clf_a,fs,'stacking_w_a')

eclf_na = VotingClassifier(estimators=estimators_na, voting='soft')
eclf_na.fit(train_x,train_y)
clf_estimate(eclf_na,fs,'voting_wo_na')

eclf_a = VotingClassifier(estimators=estimators_a, voting='soft')
eclf_a.fit(train_x,train_y)
clf_estimate(eclf_a,fs,'voting_w_a')

estimators = [('catboost_wo_na',model['catboost_wo_na']),
                   ('lightgbm_wo_na',model['lightgbm_wo_na']),
                   ('randomforest_wo_na',model['randomforest_wo_na']),
                   ('catboost_w_a',model['catboost_w_a']),
                   ('lightgbm_w_a',model['lightgbm_w_a']),
                   ('randomforest_w_a',model['randomforest_w_a'])]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(),cv=3
    )
clf.fit(train_x,train_y)
clf_estimate(clf,fs,'stacking_combined')

eclf = VotingClassifier(estimators=estimators, voting='soft')
eclf.fit(train_x,train_y)
clf_estimate(eclf,fs,'voting_combined')

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] lambda_l1 is set=0.03465726343276296, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.03465726343276296
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] min_gain_to_split is set=1.0671474959087606, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.0671474959087606
[LightGBM] [Warning] lambda_l2 is set=0.03465726343276296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.03465726343276296
[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=1200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=0.03465726343276296, reg_alpha

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

[LightGBM] [Warning] lambda_l1 is set=1.1494828601499977e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1494828601499977e-06
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] min_gain_to_split is set=0.17113215428784745, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.17113215428784745
[LightGBM] [Warning] lambda_l2 is set=1.1494828601499977e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.1494828601499977e-06
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=1700, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1700
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=1.1494828601499977e-06, reg_alpha=0.0 will be ig

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

...

[LightGBM] [Warning] lambda_l1 is set=1.1494828601499977e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1494828601499977e-06
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] min_gain_to_split is set=0.17113215428784745, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.17113215428784745
[LightGBM] [Warning] lambda_l2 is set=1.1494828601499977e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.1494828601499977e-06
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=1700, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1700
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] lambda_l1 is set=0.03465726343276296, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.03465726343276296
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] min_gain_to_split is set=1.0671474959087606, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.0671474959087606
[LightGBM] [Warning] lambda_l2 is set=0.03465726343276296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.03465726343276296
[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=1200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=0.03465726343276296, reg_alpha

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

[LightGBM] [Warning] lambda_l1 is set=0.03465726343276296, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.03465726343276296
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] min_gain_to_split is set=1.0671474959087606, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.0671474959087606
[LightGBM] [Warning] lambda_l2 is set=0.03465726343276296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.03465726343276296
[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=1200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


In [1]:
clf

NameError: name 'clf' is not defined